In [1]:
import os
import pandas as pd
import numpy as np
import re
from collections import Counter, defaultdict
from docx import Document

## Parser Evaluation

In [3]:
df1 = pd.read_excel("../data/Annotator_1.xlsx", sheet_name=None, header=1, usecols=["line","S","N","C","D","E","T","M"])
df2 = pd.read_excel("../data/Annotator_2.xlsx", sheet_name=None, header=1, usecols=["line","S","N","C","D","E","T","M"])
df3 = pd.read_excel("../data/Annotator_3.xlsx", sheet_name=None, header=1, usecols=["line","S","N","C","D","E","T","M"])

In [4]:
movies = list(df1.keys())[1:]

for movie in movies:
    for i, df in enumerate([df1, df2, df3]):
        movie_df = df[movie]
        n_lines = 0
        n_valid_lines = 0
        n_invalid_lines = 0

        for index, row in movie_df.iterrows():
            line = row["line"]

            if (isinstance(line, str) and len(line.strip()) > 0) or (isinstance(line, float) and not np.isnan(line)):
                n_matched_lines = 0
                n_nans = 0
                n_other = 0

                for col in ["S","N","C","D","E","T","M"]:
                    colline = row[col]
                    if line == colline:
                        n_matched_lines += 1
                    elif isinstance(colline, float) and np.isnan(colline):
                        n_nans += 1
                    else:
                        n_other += 1
                
                if (n_matched_lines == 1 and n_nans == 6) or n_nans == 7:
                    n_valid_lines += 1
                else:
                    # print(row)
                    n_invalid_lines += 1
                n_lines += 1
        
        print("{:35s}: Rater{}   {:3d} lines, {:3d} valid and {:3d} invalid".format(movie, i + 1, n_lines, n_valid_lines, n_invalid_lines))
        print()
    print("\n\n")

44_inch_chest                      : Rater1   251 lines, 251 valid and   0 invalid

44_inch_chest                      : Rater2   251 lines, 251 valid and   0 invalid

44_inch_chest                      : Rater3   251 lines, 251 valid and   0 invalid




american_psycho                    : Rater1   251 lines, 251 valid and   0 invalid

american_psycho                    : Rater2   251 lines, 251 valid and   0 invalid

american_psycho                    : Rater3   251 lines, 251 valid and   0 invalid




basic_instinct                     : Rater1   250 lines, 250 valid and   0 invalid

basic_instinct                     : Rater2   250 lines, 250 valid and   0 invalid

basic_instinct                     : Rater3   250 lines, 250 valid and   0 invalid




bodyguard                          : Rater1   245 lines, 245 valid and   0 invalid

bodyguard                          : Rater2   245 lines, 245 valid and   0 invalid

bodyguard                          : Rater3   245 lines, 245 valid 

In [48]:
for movie in movies:
    mlines = df3[movie]["line"].values
    flines = open("../data/screenplays/{}.txt".format(movie)).readlines()

    i, mline1 = -1, ""
    j, mline2 = -1, ""
    k, mline3 = -1, ""
    l, mlinen = -1, ""

    n = len(mlines)

    for p, mline in enumerate(mlines):
        if isinstance(mline, str) and len(mline.strip()) > 0:
            mline = mline.strip()
            if i == -1:
                i = p
                mline1 = mline
            elif j == -1:
                j = p
                mline2 = mline
            elif k == -1:
                k = p
                mline3 = mline
                break
    
    for p, mline in enumerate(reversed(mlines)):
        if isinstance(mline, str) and len(mline.strip()) > 0:
            l = n - 1 - p
            mlinen = mline.strip()
            break
    
    u, v = -1, -1

    for p, fline in enumerate(flines):
        fline = fline.strip()
        if len(fline) > 0 and fline == mline1:
            q = p + j - i
            r = p + k - i
            s = p + l - i
            if flines[q].strip() == mline2 and flines[r].strip() == mline3 and flines[s].strip() == mlinen:
                u = p - i
                v = u + n
                break
    
    print(movie, n, u, v)

44_inch_chest 310 4415 4725
american_psycho 344 712 1056
basic_instinct 334 4032 4366
bodyguard 276 2901 3177
bounty_hunter_the 396 5497 5893
broken_embraces 544 5308 5852
burn_after_reading 398 3113 3511
candle_to_water 410 2532 2942
changeling 346 7526 7872
custody 381 4404 4785
dry_white_season_a 287 6809 7096
event_horizon 360 3577 3937
extract 395 4200 4595
flight 357 4170 4527
gamer 358 2349 2707
grosse_pointe_blank 366 4026 4392
i_am_number_four 375 590 965
kids 416 2344 2760
lord_of_the_rings_fellowship_of 378 50 428
machete 402 3933 4335
man_who_knew_too_much_the 371 3476 3847
memento 375 1273 1648
men_in_black_3 379 1494 1873
mirrors 418 622 1040
nine 356 4088 4444
pandorum 392 4930 5322
petulia 388 2415 2803
shawshank_redemption_the 349 2288 2637
spartan 334 2725 3059
star_trek_01_the_motion_picture 335 2509 2844
strange_days 322 5385 5707
stuntman_the 338 2894 3232
suspect_zero 383 1439 1822
true_romance 366 1133 1499
up_in_the_air 337 5896 6233
up 478 4900 5378
willow 343 

In [51]:
ann = {}

for movie in movies:
    ann[movie] = {}
    
    for i, df in enumerate([df1, df2, df3]):
        anns = []

        for _, row in df[movie].iterrows():
            if not isinstance(row["line"], str) or (isinstance(row["line"], str) and row["line"].strip() == ""):
                anns.append("O")
            else:
                n = 0
                atag = ""
                for tag in ["S","N","C","D","E","T"]:
                    if isinstance(row[tag], str) and row["line"].strip() == row[tag].strip():
                        atag = tag
                    else:
                        n += 1
                if atag != "" and n == 5:
                    anns.append(atag)
                else:
                    anns.append("O")
        
        ann[movie]["ann{}".format(i + 1)] = anns

In [57]:
for movie in movies:
    n1, n2, n3 = 0, 0, 0
    for a1, a2, a3 in zip(ann[movie]["ann1"], ann[movie]["ann2"], ann[movie]["ann3"]):
        S = set([a1, a2, a3])
        if len(S) == 1:
            n1 += 1
        elif len(S) == 2:
            n2 += 1
        else:
            n3 += 1
    n = n1 + n2 + n3
    print("{:35s} {:3.1f}% all agree, {} two agree, {} all disagree".format(movie, 100*n1/n, n2, n3))

44_inch_chest                       92.9% all agree, 22 two agree, 0 all disagree
american_psycho                     98.8% all agree, 3 two agree, 1 all disagree
basic_instinct                      96.1% all agree, 13 two agree, 0 all disagree
bodyguard                           97.5% all agree, 7 two agree, 0 all disagree
bounty_hunter_the                   99.5% all agree, 2 two agree, 0 all disagree
broken_embraces                     97.2% all agree, 15 two agree, 0 all disagree
burn_after_reading                  99.2% all agree, 3 two agree, 0 all disagree
candle_to_water                     98.0% all agree, 8 two agree, 0 all disagree
changeling                          99.7% all agree, 1 two agree, 0 all disagree
custody                             98.2% all agree, 7 two agree, 0 all disagree
dry_white_season_a                  98.6% all agree, 4 two agree, 0 all disagree
event_horizon                       99.4% all agree, 2 two agree, 0 all disagree
extract                  

In [59]:
for movie in ann.keys():
    maj = []
    for a1, a2, a3 in zip(ann[movie]["ann1"], ann[movie]["ann2"], ann[movie]["ann3"]):
        if a1 == a2 or a1 == a3:
            maj.append(a1)
        elif a2 == a3:
            maj.append(a2)
        else:
            maj.append("O")
    ann[movie]["maj"] = maj

In [75]:
line_indices = open("../data/line_indices.txt").readlines()

for line in line_indices:
    movie, _, i, j = line.split()
    i, j = int(i), int(j)
    ann[movie]["start"] = i
    ann[movie]["end"] = j
    tags = open("../data/parsed-screenplays/{}_tags.txt".format(movie)).read().splitlines()
    tags = [t if t in ["S","N","C","D","E","T"] else "O" for t in tags]
    wtags = open("../data/parsed-leading-whitespace-removed-screenplays/{}_tags.txt".format(movie)).read().splitlines()
    wtags = [t if t in ["S","N","C","D","E","T"] else "O" for t in wtags]
    ann[movie]["sys"] = tags[i:j]
    ann[movie]["sysw"] = wtags[i:j]
    assert len(ann[movie]["maj"]) == len(ann[movie]["sys"]) == len(ann[movie]["sysw"])

In [76]:
for movie in ann.keys():
    n1 = sum(x == y for x, y in zip(ann[movie]["maj"], ann[movie]["sys"]))
    n2 = sum(x == y for x, y in zip(ann[movie]["maj"], ann[movie]["sysw"]))
    N = len(ann[movie]["maj"])
    print("{:35s} acc = {:5.1f}% acc-w = {:5.1f}%".format(movie, 100*n1/N, 100*n2/N))

44_inch_chest                       acc =  82.9% acc-w =  82.9%
american_psycho                     acc =  98.5% acc-w =  98.5%
basic_instinct                      acc = 100.0% acc-w = 100.0%
bodyguard                           acc =  74.6% acc-w =  74.6%
bounty_hunter_the                   acc =  98.7% acc-w =  98.7%
broken_embraces                     acc =  96.0% acc-w =  96.0%
burn_after_reading                  acc =  99.0% acc-w =  99.0%
candle_to_water                     acc =  95.4% acc-w =  95.4%
changeling                          acc =  98.0% acc-w =  98.0%
custody                             acc =  94.0% acc-w =  94.0%
dry_white_season_a                  acc =  92.0% acc-w =  92.0%
event_horizon                       acc = 100.0% acc-w = 100.0%
extract                             acc =  86.8% acc-w =  86.8%
flight                              acc =  98.0% acc-w =  98.0%
gamer                               acc =  95.5% acc-w =  95.5%
grosse_pointe_blank                 acc 

In [74]:
for tag in ["S","N","C","D","E","T"]:
    tp, fp, fn = 0, 0, 0
    for movie in ann.keys():
        tp += sum(x == y == tag for x, y in zip(ann[movie]["maj"], ann[movie]["sys"]))
        fp += sum(x != y == tag for x, y in zip(ann[movie]["maj"], ann[movie]["sys"]))
        fn += sum(tag == x != y for x, y in zip(ann[movie]["maj"], ann[movie]["sys"]))
    p = tp/(tp + fp)
    r = tp/(tp + fn)
    f1 = 2*p*r/(p + r)
    print("{}: p = {:4.1f}, r = {:4.1f}, f1 = {:4.1f}".format(tag, 100*p, 100*r, 100*f1))

S: p = 99.7, r = 95.9, f1 = 97.8
N: p = 89.9, r = 89.7, f1 = 89.8
C: p = 97.5, r = 98.1, f1 = 97.8
D: p = 91.6, r = 95.6, f1 = 93.5
E: p = 84.3, r = 88.3, f1 = 86.2
T: p = 86.3, r = 92.6, f1 = 89.4


In [77]:
for tag in ["S","N","C","D","E","T"]:
    tp, fp, fn = 0, 0, 0
    for movie in ann.keys():
        tp += sum(x == y == tag for x, y in zip(ann[movie]["maj"], ann[movie]["sysw"]))
        fp += sum(x != y == tag for x, y in zip(ann[movie]["maj"], ann[movie]["sysw"]))
        fn += sum(tag == x != y for x, y in zip(ann[movie]["maj"], ann[movie]["sysw"]))
    p = tp/(tp + fp)
    r = tp/(tp + fn)
    f1 = 2*p*r/(p + r)
    print("{}: p = {:4.1f}, r = {:4.1f}, f1 = {:4.1f}".format(tag, 100*p, 100*r, 100*f1))

S: p = 99.7, r = 95.9, f1 = 97.8
N: p = 89.9, r = 89.7, f1 = 89.8
C: p = 97.5, r = 98.1, f1 = 97.8
D: p = 91.6, r = 95.6, f1 = 93.5
E: p = 84.3, r = 88.3, f1 = 86.2
T: p = 86.3, r = 92.6, f1 = 89.4


## GDI Scripts

In [21]:
folders = ["LEGO TITAN", "Lionsgate", "NBC Universal"]

data = []

for folder in folders:
    movies = sorted([re.sub("\.txt$", "", f) for f in os.listdir("../data/SAIL Team Spellcheck/{}/Scripts_Txt/".format(folder)) if f.endswith(".txt")])

    for movie in movies:
        analysis_docx = os.path.join("../data/SAIL Team Spellcheck/{}/Analysis/{}.docx".format(folder, movie))
        
        if os.path.exists(analysis_docx):
            print(folder, movie)

            
            with open(analysis_docx, "rb") as f:
                doc = Document(f)
                table = doc.tables[0]

                i = 2
                cells = [cell.text for row in table.rows for cell in row.cells]

                while i + 1 < len(cells):
                     data.append([folder, movie, cells[i], cells[i + 1]])
                     i += 2

line_count_df = pd.DataFrame(data, columns=["folder", "movie", "character", "line-count"])

LEGO TITAN EPISODE 102
LEGO TITAN EPISODE 103
LEGO TITAN EPISODE 105
LEGO TITAN EPISODE 107
LEGO TITAN EPISODE 108
LEGO TITAN EPISODE 109
LEGO TITAN EPISODE 110
LEGO TITAN EPISODE 111
LEGO TITAN EPISODE 112
LEGO TITAN EPISODE 114
LEGO TITAN EPISODE 116
LEGO TITAN EPISODE 117
Lionsgate 16 States
Lionsgate ABOUT MY FATHER
Lionsgate AMERICAN UNDERDOG
Lionsgate ANGELS
Lionsgate ARE YOU THERE GOD, IT_s ME MARGARET
Lionsgate Bad Dog
Lionsgate Borderlands
Lionsgate DIRTY DANCING
Lionsgate Escape
Lionsgate Free Agents
Lionsgate Joy Fuck Club
Lionsgate London
Lionsgate MAGIC _ BIRD
Lionsgate MEET ME IN NEW DELHI
Lionsgate MillersGirl
Lionsgate NARUTO
Lionsgate OpenWater
Lionsgate SHADOW FORCE
Lionsgate SHOTGUN WEDDING
Lionsgate THE SUN ALWAYS SETS IN THE WEST
Lionsgate THE VALET
Lionsgate TIGERBELLES
Lionsgate Unbreakable
Lionsgate WHITEBIRD
NBC Universal BEAST_Spellcheck
NBC Universal BRONZEVILLE
NBC Universal FIRESTARTER
NBC Universal KID SWAP
NBC Universal PUDGE PATROL
NBC Universal Small Da

In [22]:
print(line_count_df.shape)
print(line_count_df)

(1129, 4)
             folder           movie     character line-count
0        LEGO TITAN     EPISODE 102         MATEO         56
1        LEGO TITAN     EPISODE 102         MR OZ         55
2        LEGO TITAN     EPISODE 102         IZZIE         47
3        LEGO TITAN     EPISODE 102        COOPER         26
4        LEGO TITAN     EPISODE 102  NIGHT HUNTER         14
...             ...             ...           ...        ...
1124  NBC Universal  THE GRACE YEAR         Tammy          5
1125  NBC Universal  THE GRACE YEAR      Gertrude         79
1126  NBC Universal  THE GRACE YEAR    Tall Guard          6
1127  NBC Universal  THE GRACE YEAR         Ryker        166
1128  NBC Universal  THE GRACE YEAR        Anders         17

[1129 rows x 4 columns]


In [31]:
line_count_df.to_csv("../data/SAIL Team Spellcheck/line_counts.tsv", sep="\t", index=False)

In [26]:
items = []

for _, row in line_count_df[["folder", "movie"]].drop_duplicates().iterrows():
    items.append((row["folder"], row["movie"]))

In [38]:
data = []

for folder, movie in items:
    slines = open("../data/SAIL Team Spellcheck/{}/Scripts_Txt/{}.txt".format(folder, movie)).read().splitlines()
    tags = open("../data/SAIL Team Spellcheck/{}/Parsed/{}_tags.txt".format(folder, movie)).read().splitlines()
    tags = [t if t in ["S","N","C","D","E","T"] else "O" for t in tags]

    characters = []
    n = len(slines)
    i = 0

    while i < n:
        if tags[i] == "C":
            character = re.sub("(^[\s\d\*]*)|((\(.+\)\s*)*[\s\d\*]*$)", "", slines[i])

            if len(character) > 2:
                i += 1
                while i < n and tags[i] in ["O","D","E"]:
                    if tags[i] == "D":
                        characters.append(character)
                    i += 1
                i -= 1
        i += 1

    citems = sorted(Counter(characters).items(), key = lambda x: x[1], reverse=True)
    for ch, cn in citems:
        data.append([folder, movie, ch, cn])

sys_line_count_df = pd.DataFrame(data, columns=["folder", "movie", "character", "line-count"])
sys_line_count_df.to_csv("../data/SAIL Team Spellcheck/sys_line_counts.tsv", sep="\t", index=False)

In [55]:
k_list = [5, 10, 20, np.inf]

for k in k_list:

    tp, fp, fn = 0, 0, 0
    errors = []
    
    for folder, movie in items:

        gitems = []
        for _, row in line_count_df.loc[(line_count_df["folder"] == folder) & (line_count_df["movie"] == movie), ["character", "line-count"]].iterrows():
            gitems.append((row["character"], int(row["line-count"])))
        gitems = sorted(gitems, key=lambda citem: citem[1], reverse=True)
        if not np.isinf(k):
            gitems = gitems[:k]
        cdict = {}
        for ch, cn in gitems:
            cdict[ch] = cn

        citems = []
        for _, row in sys_line_count_df.loc[(sys_line_count_df["folder"] == folder) & (sys_line_count_df["movie"] == movie), ["character", "line-count"]].iterrows():
            citems.append((row["character"], int(row["line-count"])))
        citems = sorted(citems, key=lambda citem: citem[1], reverse=True)
        if not np.isinf(k):
            citems = citems[:k]
        sys_cdict = {}
        for ch, cn in citems:
            sys_cdict[ch] = cn
        
        characters = set(list(cdict.keys()))
        sys_characters = set(list(sys_cdict.keys()))
        common_characters = characters.intersection(sys_characters)

        tp += len(common_characters)
        fp += len(sys_characters.difference(characters))
        fn += len(characters.difference(sys_characters))

        for character in common_characters:
            errors.append(sys_cdict[character] - cdict[character])
    
    p = tp/(tp + fp)
    r = tp/(tp + fn)
    f1 = 2 * p * r / (p + r)
    mae = np.mean(np.absolute(errors))
    rmse = np.sqrt(np.mean(np.square(errors)))

    print("k = {:5s}::: character identification: precision = {:6.2f}, recall = {:6.2f}, F1 = {:6.2f} :: line count error: MAE = {:6.2f}, RMSE = {:6.2f}".format(str(k), p, r, f1, mae, rmse))

k = 5    ::: character identification: precision =   0.59, recall =   0.59, F1 =   0.59 :: line count error: MAE =  59.06, RMSE = 193.93
k = 10   ::: character identification: precision =   0.56, recall =   0.57, F1 =   0.56 :: line count error: MAE =  42.22, RMSE = 149.32
k = 20   ::: character identification: precision =   0.46, recall =   0.53, F1 =   0.49 :: line count error: MAE =  30.83, RMSE = 119.02
k = inf  ::: character identification: precision =   0.31, recall =   0.65, F1 =   0.42 :: line count error: MAE =  18.23, RMSE =  87.46
